In [ ]:
# 1
# Chance Wiese

In [6]:
# 2
print('''
Name: Puptrail, Inc.
Product: “teardrop” camper trailers
Locations:
    1. Salt Lake City
    2. Phoenix
    3. Boise
''')


Company Name: Puptrail, Inc.
Company Product: “teardrop” camper trailers
Locations:
    1. Salt Lake City
    2. Phoenix
    3. Boise



In [ ]:
# 3
import pandas as pd
import numpy as np
import psycopg2
import pymssql

conn = psycopg2.connect(
    database="puptrail", 
    user='dw_chancewiese',
    password='Spikeball2020',
    host='database-1.czsooswggscz.us-east-2.rds.amazonaws.com',
    port='5432'
)

cursor = conn.cursor()

query = '''
SELECT 
    (o.builtdate - o.promisedate) AS delay_days,
    COUNT(*) AS num_trailers,
    AVG(s.delsatisfac) AS avg_satisfaction
FROM oline o
JOIN salesorder s ON o.orderid = s.orderid
JOIN product p ON o.productid = p.prodid
JOIN prodcat pc ON p.catid = pc.catid
WHERE pc.category = 'Trailer'
AND o.builtdate > o.promisedate
GROUP BY delay_days
ORDER BY delay_days;
'''

df = pd.read_sql(query, conn)
print(df)

